In [1]:
%cp /content/drive/MyDrive/QA/evaluate-v2.0.py /content/
%cp /content/drive/MyDrive/QA/dev-v2.0.json /content/

In [2]:
%%writefile EnsembleClassifier.py

import numpy as np


class EnsembleClassifier():


	def __init__(self, type):
		self.type = type


	def predict1(self, B_op, A_op, R_op):
		final_op = {}
		final_voter = {}
		qIds = list(B_op.keys())
		for qId in qIds:
			votes = [0,0,0]
			if (qId in B_op):
				votes[0] +=1
			if (qId in A_op):
				votes[1] +=1
			if (qId in R_op):
				votes[2] +=1
			if ((qId in B_op) and (qId in A_op) and (B_op[qId]==A_op[qId])): 
				votes[0] +=1
				votes[1] +=1
			if ((qId in B_op) and (qId in R_op) and (B_op[qId]==R_op[qId])): 
				votes[0] +=1
				votes[2] +=1
			if ((qId in A_op) and (qId in R_op) and (A_op[qId]==R_op[qId])): 
				votes[1] +=1
				votes[2] +=1
			votesmaxindex = np.argmax(votes)
			final_op[qId] = B_op[qId] if votesmaxindex==0 else (A_op[qId] if votesmaxindex==1 else R_op[qId])
			final_voter[qId] =  { "maxVotes": "B" if votesmaxindex==0 else ("A" if votesmaxindex==1 else "R"),
									"votes": votes }
		return final_op, final_voter


	def predict2(self, B_op, A_op, R_op):
		final_op = {}
		final_voter = {}
		qIds = list(B_op.keys())
		for qId in qIds:
			votes = [0,0,0]
			isNonAnswerable = False
			temp_cnt = 0
			if (qId in B_op):
				votes[0] +=1
			if (qId in A_op):
				votes[1] +=1
			if (qId in R_op):
				votes[2] +=1
			if ((qId in B_op) and (qId in A_op) and (B_op[qId]==A_op[qId])): 
				votes[0] +=1
				votes[1] +=1
			if ((qId in B_op) and (qId in R_op) and (B_op[qId]==R_op[qId])): 
				votes[0] +=1
				votes[2] +=1
			if ((qId in A_op) and (qId in R_op) and (A_op[qId]==R_op[qId])): 
				votes[1] +=1
				votes[2] +=1
			votesmaxindex = np.argmax(votes)
			final_op[qId] = B_op[qId] if votesmaxindex==0 else (A_op[qId] if votesmaxindex==1 else R_op[qId])
			if (final_op[qId]!=""):
				final_op[qId] = B_op[qId]
				final_voter[qId] =  { "maxVotes": "B", "votes": votes }
			else:
				final_voter[qId] =  { "maxVotes": "B" if votesmaxindex==0 else ("A" if votesmaxindex==1 else "R"),
									"votes": votes }
		return final_op, final_voter

Writing EnsembleClassifier.py


In [ ]:
%%writefile Squad_v2.py

import pandas as pd
import numpy as np
import json
from EnsembleClassifier import EnsembleClassifier


devfile_location = '/content/drive/MyDrive/QA/dev-v2.0.json'
B_opfile_location = '/content/drive/MyDrive/QA/model_results/bert-base-cased/squad_v2/eval_predictions.json'
A_opfile_location = '/content/drive/MyDrive/QA/model_results/albert-base-v2/squad_v2/eval_predictions.json'
R_opfile_location = '/content/drive/MyDrive/QA/model_results/roberta-base/squad_v2/eval_predictions.json'


def fetchDevData():
	with open(B_opfile_location) as json_data:
	    return json.load(json_data)


def fetchModelsOP():
	with open(B_opfile_location) as json_data:
	    B_op = json.load(json_data)
	with open(A_opfile_location) as json_data:
	    A_op = json.load(json_data)
	with open(R_opfile_location) as json_data:
	    R_op = json.load(json_data)
	return B_op, A_op, R_op


def writeJSON(data, fileName):
	with open(fileName+'.json', 'w') as outfile:
		json.dump(data, outfile)


def main():
	data = fetchDevData()
	B_op, A_op, R_op = fetchModelsOP()
	print("Starting Ensemble Classifier: ")
	ec = EnsembleClassifier("MAX_VOTING")
	final_op1, final_voter1 = ec.predict1(B_op, A_op, R_op)
	writeJSON(final_op1, "squad_v2_final_predictions1")
	writeJSON(final_voter1, "squad_v2_final_predictionsVoter1")
	print("Predictions written to file final_predictions1.json")
	final_op2, final_voter2 = ec.predict2(B_op, A_op, R_op)
	writeJSON(final_op2, "squad_v2_final_predictions2")
	writeJSON(final_voter2, "squad_v2_final_predictionsVoter2")
	print("Predictions written to file final_predictions2.json")


main()

Writing Squad_v2.py


In [ ]:
!python Squad_v2.py

Starting Ensemble Classifier: 
Predictions written to file final_predictions1.json
Predictions written to file final_predictions2.json


In [ ]:
%cp /content/squad_v2_final_predictions1.json /content/drive/MyDrive/QA/model_results
%cp /content/squad_v2_final_predictions2.json /content/drive/MyDrive/QA/model_results
%cp /content/squad_v2_final_predictionsVoter1.json /content/drive/MyDrive/QA/model_results
%cp /content/squad_v2_final_predictionsVoter2.json /content/drive/MyDrive/QA/model_results

In [ ]:
!python evaluate-v2.0.py dev-v2.0.json squad_v2_final_predictions1.json

{
  "exact": 79.55024004042787,
  "f1": 82.50383338224069,
  "total": 11873,
  "HasAns_exact": 77.32793522267207,
  "HasAns_f1": 83.24359206264239,
  "HasAns_total": 5928,
  "NoAns_exact": 81.76619007569386,
  "NoAns_f1": 81.76619007569386,
  "NoAns_total": 5945
}


In [ ]:
!python evaluate-v2.0.py dev-v2.0.json squad_v2_final_predictions2.json

{
  "exact": 77.38566495409754,
  "f1": 80.72644958510205,
  "total": 11873,
  "HasAns_exact": 71.7442645074224,
  "HasAns_f1": 78.43541429215863,
  "HasAns_total": 5928,
  "NoAns_exact": 83.01093355761144,
  "NoAns_f1": 83.01093355761144,
  "NoAns_total": 5945
}


In [ ]:
%%writefile Back-trans-aug.py

import pandas as pd
import numpy as np
import json
from EnsembleClassifier import EnsembleClassifier


devfile_location = '/content/drive/MyDrive/QA/dev-v2.0.json'
B_opfile_location = '/content/drive/MyDrive/QA/model_results/bert-base-cased/back-trans-aug/eval_predictions.json'
A_opfile_location = '/content/drive/MyDrive/QA/model_results/albert-base-v2/back-trans-aug/eval_predictions.json'
R_opfile_location = '/content/drive/MyDrive/QA/model_results/roberta-base/back-trans-aug/eval_predictions.json'


def fetchDevData():
	with open(B_opfile_location) as json_data:
	    return json.load(json_data)


def fetchModelsOP():
	with open(B_opfile_location) as json_data:
	    B_op = json.load(json_data)
	with open(A_opfile_location) as json_data:
	    A_op = json.load(json_data)
	with open(R_opfile_location) as json_data:
	    R_op = json.load(json_data)
	return B_op, A_op, R_op


def writeJSON(data, fileName):
	with open(fileName+'.json', 'w') as outfile:
		json.dump(data, outfile)


def main():
	data = fetchDevData()
	B_op, A_op, R_op = fetchModelsOP()
	print("Starting Ensemble Classifier: ")
	ec = EnsembleClassifier("MAX_VOTING")
	final_op1, final_voter1 = ec.predict1(B_op, A_op, R_op)
	writeJSON(final_op1, "back-trans-aug_final_predictions1")
	writeJSON(final_voter1, "back-trans-aug_final_predictionsVoter1")
	print("Predictions written to file final_predictions1.json")
	final_op2, final_voter2 = ec.predict2(B_op, A_op, R_op)
	writeJSON(final_op2, "back-trans-aug_final_predictions2")
	writeJSON(final_voter2, "back-trans-aug_final_predictionsVoter2")
	print("Predictions written to file final_predictions2.json")


main()

Writing Back-trans-aug.py


In [ ]:
!python Back-trans-aug.py

Starting Ensemble Classifier: 
Predictions written to file final_predictions1.json
Predictions written to file final_predictions2.json


In [ ]:
%cp /content/back-trans-aug_final_predictions1.json /content/drive/MyDrive/QA/model_results
%cp /content/back-trans-aug_final_predictions2.json /content/drive/MyDrive/QA/model_results
%cp /content/back-trans-aug_final_predictionsVoter1.json /content/drive/MyDrive/QA/model_results
%cp /content/back-trans-aug_final_predictionsVoter2.json /content/drive/MyDrive/QA/model_results

In [ ]:
!python evaluate-v2.0.py dev-v2.0.json back-trans-aug_final_predictions1.json

{
  "exact": 78.68272551166513,
  "f1": 81.48228389283136,
  "total": 11873,
  "HasAns_exact": 77.26045883940621,
  "HasAns_f1": 82.86760402489664,
  "HasAns_total": 5928,
  "NoAns_exact": 80.10092514718251,
  "NoAns_f1": 80.10092514718251,
  "NoAns_total": 5945
}


In [ ]:
!python evaluate-v2.0.py dev-v2.0.json back-trans-aug_final_predictions2.json

{
  "exact": 76.29916617535585,
  "f1": 79.57586697845448,
  "total": 11873,
  "HasAns_exact": 71.17071524966262,
  "HasAns_f1": 77.73351360242752,
  "HasAns_total": 5928,
  "NoAns_exact": 81.41295206055509,
  "NoAns_f1": 81.41295206055509,
  "NoAns_total": 5945
}


In [ ]:
%%writefile Back-trans-possible-aug.py

import pandas as pd
import numpy as np
import json
from EnsembleClassifier import EnsembleClassifier


devfile_location = '/content/drive/MyDrive/QA/dev-v2.0.json'
B_opfile_location = '/content/drive/MyDrive/QA/model_results/bert-base-cased/back-trans-possible-aug/eval_predictions.json'
A_opfile_location = '/content/drive/MyDrive/QA/model_results/albert-base-v2/back-trans-possible-aug/eval_predictions.json'
R_opfile_location = '/content/drive/MyDrive/QA/model_results/roberta-base/back-trans-possible-aug/eval_predictions.json'


def fetchDevData():
	with open(B_opfile_location) as json_data:
	    return json.load(json_data)


def fetchModelsOP():
	with open(B_opfile_location) as json_data:
	    B_op = json.load(json_data)
	with open(A_opfile_location) as json_data:
	    A_op = json.load(json_data)
	with open(R_opfile_location) as json_data:
	    R_op = json.load(json_data)
	return B_op, A_op, R_op


def writeJSON(data, fileName):
	with open(fileName+'.json', 'w') as outfile:
		json.dump(data, outfile)


def main():
	data = fetchDevData()
	B_op, A_op, R_op = fetchModelsOP()
	print("Starting Ensemble Classifier: ")
	ec = EnsembleClassifier("MAX_VOTING")
	final_op1, final_voter1 = ec.predict1(B_op, A_op, R_op)
	writeJSON(final_op1, "back-trans-possible-aug_final_predictions1")
	writeJSON(final_voter1, "back-trans-possible-aug_final_predictionsVoter1")
	print("Predictions written to file final_predictions1.json")
	final_op2, final_voter2 = ec.predict2(B_op, A_op, R_op)
	writeJSON(final_op2, "back-trans-possible-aug_final_predictions2")
	writeJSON(final_voter2, "back-trans-possible-aug_final_predictionsVoter2")
	print("Predictions written to file final_predictions2.json")


main()

Writing Back-trans-possible-aug.py


In [ ]:
!python Back-trans-possible-aug.py

Starting Ensemble Classifier: 
Predictions written to file final_predictions1.json
Predictions written to file final_predictions2.json


In [ ]:
%cp /content/back-trans-possible-aug_final_predictions1.json /content/drive/MyDrive/QA/model_results
%cp /content/back-trans-possible-aug_final_predictions2.json /content/drive/MyDrive/QA/model_results
%cp /content/back-trans-possible-aug_final_predictionsVoter1.json /content/drive/MyDrive/QA/model_results
%cp /content/back-trans-possible-aug_final_predictionsVoter2.json /content/drive/MyDrive/QA/model_results

In [ ]:
!python evaluate-v2.0.py dev-v2.0.json back-trans-possible-aug_final_predictions1.json

{
  "exact": 78.50585361745136,
  "f1": 81.37150448836555,
  "total": 11873,
  "HasAns_exact": 77.05802968960863,
  "HasAns_f1": 82.79754939108707,
  "HasAns_total": 5928,
  "NoAns_exact": 79.94953742640875,
  "NoAns_f1": 79.94953742640875,
  "NoAns_total": 5945
}


In [ ]:
!python evaluate-v2.0.py dev-v2.0.json back-trans-possible-aug_final_predictions2.json

{
  "exact": 76.23178640613156,
  "f1": 79.51153230323797,
  "total": 11873,
  "HasAns_exact": 71.10323886639677,
  "HasAns_f1": 77.67213613973423,
  "HasAns_total": 5928,
  "NoAns_exact": 81.34566862910009,
  "NoAns_f1": 81.34566862910009,
  "NoAns_total": 5945
}


In [ ]:
%%writefile Back-trans-synonym-aug.py

import pandas as pd
import numpy as np
import json
from EnsembleClassifier import EnsembleClassifier


devfile_location = '/content/drive/MyDrive/QA/dev-v2.0.json'
B_opfile_location = '/content/drive/MyDrive/QA/model_results/bert-base-cased/back-trans-synonym-aug/eval_predictions.json'
A_opfile_location = '/content/drive/MyDrive/QA/model_results/albert-base-v2/back-trans-synonym-aug/eval_predictions.json'
R_opfile_location = '/content/drive/MyDrive/QA/model_results/roberta-base/back-trans-synonym-aug/eval_predictions.json'


def fetchDevData():
	with open(B_opfile_location) as json_data:
	    return json.load(json_data)


def fetchModelsOP():
	with open(B_opfile_location) as json_data:
	    B_op = json.load(json_data)
	with open(A_opfile_location) as json_data:
	    A_op = json.load(json_data)
	with open(R_opfile_location) as json_data:
	    R_op = json.load(json_data)
	return B_op, A_op, R_op


def writeJSON(data, fileName):
	with open(fileName+'.json', 'w') as outfile:
		json.dump(data, outfile)


def main():
	data = fetchDevData()
	B_op, A_op, R_op = fetchModelsOP()
	print("Starting Ensemble Classifier: ")
	ec = EnsembleClassifier("MAX_VOTING")
	final_op1, final_voter1 = ec.predict1(B_op, A_op, R_op)
	writeJSON(final_op1, "back-trans-synonym-aug_final_predictions1")
	writeJSON(final_voter1, "back-trans-synonym-aug_final_predictionsVoter1")
	print("Predictions written to file final_predictions1.json")
	final_op2, final_voter2 = ec.predict2(B_op, A_op, R_op)
	writeJSON(final_op2, "back-trans-synonym-aug_final_predictions2")
	writeJSON(final_voter2, "back-trans-synonym-aug_final_predictionsVoter2")
	print("Predictions written to file final_predictions2.json")


main()

Writing Back-trans-synonym-aug.py


In [ ]:
!python Back-trans-synonym-aug.py

Starting Ensemble Classifier: 
Predictions written to file final_predictions1.json
Predictions written to file final_predictions2.json


In [ ]:
%cp /content/back-trans-synonym-aug_final_predictions1.json /content/drive/MyDrive/QA/model_results
%cp /content/back-trans-synonym-aug_final_predictions2.json /content/drive/MyDrive/QA/model_results
%cp /content/back-trans-synonym-aug_final_predictionsVoter1.json /content/drive/MyDrive/QA/model_results
%cp /content/back-trans-synonym-aug_final_predictionsVoter2.json /content/drive/MyDrive/QA/model_results

In [ ]:
!python evaluate-v2.0.py dev-v2.0.json back-trans-synonym-aug_final_predictions1.json

{
  "exact": 75.76855049271457,
  "f1": 78.58977876058432,
  "total": 11873,
  "HasAns_exact": 78.03643724696356,
  "HasAns_f1": 83.68698434959818,
  "HasAns_total": 5928,
  "NoAns_exact": 73.5071488645921,
  "NoAns_f1": 73.5071488645921,
  "NoAns_total": 5945
}


In [ ]:
!python evaluate-v2.0.py dev-v2.0.json back-trans-synonym-aug_final_predictions2.json

{
  "exact": 75.76855049271457,
  "f1": 78.58977876058432,
  "total": 11873,
  "HasAns_exact": 78.03643724696356,
  "HasAns_f1": 83.68698434959818,
  "HasAns_total": 5928,
  "NoAns_exact": 73.5071488645921,
  "NoAns_f1": 73.5071488645921,
  "NoAns_total": 5945
}


In [ ]:
%%writefile Back-trans-synonym-possible-aug.py

import pandas as pd
import numpy as np
import json
from EnsembleClassifier import EnsembleClassifier


devfile_location = '/content/drive/MyDrive/QA/dev-v2.0.json'
B_opfile_location = '/content/drive/MyDrive/QA/model_results/bert-base-cased/back-trans-synonym-possible-aug/eval_predictions.json'
A_opfile_location = '/content/drive/MyDrive/QA/model_results/albert-base-v2/back-trans-synonym-possible-aug/eval_predictions.json'
R_opfile_location = '/content/drive/MyDrive/QA/model_results/roberta-base/back-trans-synonym-possible-aug/eval_predictions.json'


def fetchDevData():
	with open(B_opfile_location) as json_data:
	    return json.load(json_data)


def fetchModelsOP():
	with open(B_opfile_location) as json_data:
	    B_op = json.load(json_data)
	with open(A_opfile_location) as json_data:
	    A_op = json.load(json_data)
	with open(R_opfile_location) as json_data:
	    R_op = json.load(json_data)
	return B_op, A_op, R_op


def writeJSON(data, fileName):
	with open(fileName+'.json', 'w') as outfile:
		json.dump(data, outfile)


def main():
	data = fetchDevData()
	B_op, A_op, R_op = fetchModelsOP()
	print("Starting Ensemble Classifier: ")
	ec = EnsembleClassifier("MAX_VOTING")
	final_op1, final_voter1 = ec.predict1(B_op, A_op, R_op)
	writeJSON(final_op1, "back-trans-synonym-possible-aug_final_predictions1")
	writeJSON(final_voter1, "back-trans-synonym-possible-aug_final_predictionsVoter1")
	print("Predictions written to file final_predictions1.json")
	final_op2, final_voter2 = ec.predict2(B_op, A_op, R_op)
	writeJSON(final_op2, "back-trans-synonym-possible-aug_final_predictions2")
	writeJSON(final_voter2, "back-trans-synonym-possible-aug_final_predictionsVoter2")
	print("Predictions written to file final_predictions2.json")


main()

Writing Back-trans-synonym-possible-aug.py


In [ ]:
!python Back-trans-synonym-possible-aug.py

Starting Ensemble Classifier: 
Predictions written to file final_predictions1.json
Predictions written to file final_predictions2.json


In [ ]:
%cp /content/back-trans-synonym-possible-aug_final_predictions1.json /content/drive/MyDrive/QA/model_results
%cp /content/back-trans-synonym-possible-aug_final_predictions2.json /content/drive/MyDrive/QA/model_results
%cp /content/back-trans-synonym-possible-aug_final_predictionsVoter1.json /content/drive/MyDrive/QA/model_results
%cp /content/back-trans-synonym-possible-aug_final_predictionsVoter2.json /content/drive/MyDrive/QA/model_results

In [ ]:
!python evaluate-v2.0.py dev-v2.0.json back-trans-synonym-possible-aug_final_predictions1.json

{
  "exact": 75.76012802156153,
  "f1": 78.60135449079756,
  "total": 11873,
  "HasAns_exact": 77.66531713900135,
  "HasAns_f1": 83.35591799413623,
  "HasAns_total": 5928,
  "NoAns_exact": 73.86038687973087,
  "NoAns_f1": 73.86038687973087,
  "NoAns_total": 5945
}


In [ ]:
!python evaluate-v2.0.py dev-v2.0.json back-trans-synonym-possible-aug_final_predictions2.json

{
  "exact": 73.71346753137371,
  "f1": 76.9419902375788,
  "total": 11873,
  "HasAns_exact": 71.60931174089069,
  "HasAns_f1": 78.07561573731002,
  "HasAns_total": 5928,
  "NoAns_exact": 75.81160639192599,
  "NoAns_f1": 75.81160639192599,
  "NoAns_total": 5945
}


In [ ]:
%%writefile Context-aug.py

import pandas as pd
import numpy as np
import json
from EnsembleClassifier import EnsembleClassifier


devfile_location = '/content/drive/MyDrive/QA/dev-v2.0.json'
B_opfile_location = '/content/drive/MyDrive/QA/model_results/bert-base-cased/context-aug/eval_predictions.json'
A_opfile_location = '/content/drive/MyDrive/QA/model_results/albert-base-v2/context-aug/eval_predictions.json'
R_opfile_location = '/content/drive/MyDrive/QA/model_results/roberta-base/context-aug/eval_predictions.json'


def fetchDevData():
	with open(B_opfile_location) as json_data:
	    return json.load(json_data)


def fetchModelsOP():
	with open(B_opfile_location) as json_data:
	    B_op = json.load(json_data)
	with open(A_opfile_location) as json_data:
	    A_op = json.load(json_data)
	with open(R_opfile_location) as json_data:
	    R_op = json.load(json_data)
	return B_op, A_op, R_op


def writeJSON(data, fileName):
	with open(fileName+'.json', 'w') as outfile:
		json.dump(data, outfile)


def main():
	data = fetchDevData()
	B_op, A_op, R_op = fetchModelsOP()
	print("Starting Ensemble Classifier: ")
	ec = EnsembleClassifier("MAX_VOTING")
	final_op1, final_voter1 = ec.predict1(B_op, A_op, R_op)
	writeJSON(final_op1, "context-aug_final_predictions1")
	writeJSON(final_voter1, "context-aug_final_predictionsVoter1")
	print("Predictions written to file final_predictions1.json")
	final_op2, final_voter2 = ec.predict2(B_op, A_op, R_op)
	writeJSON(final_op2, "context-aug_final_predictions2")
	writeJSON(final_voter2, "context-aug_final_predictionsVoter2")
	print("Predictions written to file final_predictions2.json")


main()

Writing Context-aug.py


In [ ]:
!python Context-aug.py

Starting Ensemble Classifier: 
Predictions written to file final_predictions1.json
Predictions written to file final_predictions2.json


In [ ]:
%cp /content/context-aug_final_predictions1.json /content/drive/MyDrive/QA/model_results
%cp /content/context-aug_final_predictions2.json /content/drive/MyDrive/QA/model_results
%cp /content/context-aug_final_predictionsVoter1.json /content/drive/MyDrive/QA/model_results
%cp /content/context-aug_final_predictionsVoter2.json /content/drive/MyDrive/QA/model_results

In [ ]:
!python evaluate-v2.0.py dev-v2.0.json context-aug_final_predictions1.json

{
  "exact": 65.4762907437042,
  "f1": 68.43714866717826,
  "total": 11873,
  "HasAns_exact": 75.91093117408907,
  "HasAns_f1": 81.84113801035866,
  "HasAns_total": 5928,
  "NoAns_exact": 55.071488645920944,
  "NoAns_f1": 55.071488645920944,
  "NoAns_total": 5945
}


In [ ]:
!python evaluate-v2.0.py dev-v2.0.json context-aug_final_predictions2.json

{
  "exact": 63.581234734271035,
  "f1": 66.98502259619617,
  "total": 11873,
  "HasAns_exact": 70.31039136302294,
  "HasAns_f1": 77.12772828688183,
  "HasAns_total": 5928,
  "NoAns_exact": 56.871320437342305,
  "NoAns_f1": 56.871320437342305,
  "NoAns_total": 5945
}


In [ ]:
%%writefile Synonym-aug.py

import pandas as pd
import numpy as np
import json
from EnsembleClassifier import EnsembleClassifier


devfile_location = '/content/drive/MyDrive/QA/dev-v2.0.json'
B_opfile_location = '/content/drive/MyDrive/QA/model_results/bert-base-cased/synonym-aug/eval_predictions.json'
A_opfile_location = '/content/drive/MyDrive/QA/model_results/albert-base-v2/synonym-aug/eval_predictions.json'
R_opfile_location = '/content/drive/MyDrive/QA/model_results/roberta-base/synonym-aug/eval_predictions.json'


def fetchDevData():
	with open(B_opfile_location) as json_data:
	    return json.load(json_data)


def fetchModelsOP():
	with open(B_opfile_location) as json_data:
	    B_op = json.load(json_data)
	with open(A_opfile_location) as json_data:
	    A_op = json.load(json_data)
	with open(R_opfile_location) as json_data:
	    R_op = json.load(json_data)
	return B_op, A_op, R_op


def writeJSON(data, fileName):
	with open(fileName+'.json', 'w') as outfile:
		json.dump(data, outfile)


def main():
	data = fetchDevData()
	B_op, A_op, R_op = fetchModelsOP()
	print("Starting Ensemble Classifier: ")
	ec = EnsembleClassifier("MAX_VOTING")
	final_op1, final_voter1 = ec.predict1(B_op, A_op, R_op)
	writeJSON(final_op1, "synonym-aug_final_predictions1")
	writeJSON(final_voter1, "synonym-aug_final_predictionsVoter1")
	print("Predictions written to file final_predictions1.json")
	final_op2, final_voter2 = ec.predict2(B_op, A_op, R_op)
	writeJSON(final_op2, "synonym-aug_final_predictions2")
	writeJSON(final_voter2, "synonym-aug_final_predictionsVoter2")
	print("Predictions written to file final_predictions2.json")


main()

Writing Synonym-aug.py


In [ ]:
!python Synonym-aug.py

Starting Ensemble Classifier: 
Predictions written to file final_predictions1.json
Predictions written to file final_predictions2.json


In [ ]:
%cp /content/synonym-aug_final_predictions1.json /content/drive/MyDrive/QA/model_results
%cp /content/synonym-aug_final_predictions2.json /content/drive/MyDrive/QA/model_results
%cp /content/synonym-aug_final_predictionsVoter1.json /content/drive/MyDrive/QA/model_results
%cp /content/synonym-aug_final_predictionsVoter2.json /content/drive/MyDrive/QA/model_results

In [ ]:
!python evaluate-v2.0.py dev-v2.0.json synonym-aug_final_predictions1.json

{
  "exact": 77.29301777141413,
  "f1": 80.1474949499798,
  "total": 11873,
  "HasAns_exact": 77.7327935222672,
  "HasAns_f1": 83.44993379573384,
  "HasAns_total": 5928,
  "NoAns_exact": 76.85449957947856,
  "NoAns_f1": 76.85449957947856,
  "NoAns_total": 5945
}


In [ ]:
!python evaluate-v2.0.py dev-v2.0.json synonym-aug_final_predictions2.json

{
  "exact": 75.13686515623684,
  "f1": 78.45216833456632,
  "total": 11873,
  "HasAns_exact": 71.64304993252361,
  "HasAns_f1": 78.28316373756877,
  "HasAns_total": 5928,
  "NoAns_exact": 78.62068965517241,
  "NoAns_f1": 78.62068965517241,
  "NoAns_total": 5945
}


In [ ]:
%%writefile Word2vec-aug.py

import pandas as pd
import numpy as np
import json
from EnsembleClassifier import EnsembleClassifier


devfile_location = '/content/drive/MyDrive/QA/dev-v2.0.json'
B_opfile_location = '/content/drive/MyDrive/QA/model_results/bert-base-cased/word2vec-aug/eval_predictions.json'
A_opfile_location = '/content/drive/MyDrive/QA/model_results/albert-base-v2/word2vec-aug/eval_predictions.json'
R_opfile_location = '/content/drive/MyDrive/QA/model_results/roberta-base/word2vec-aug/eval_predictions.json'


def fetchDevData():
	with open(B_opfile_location) as json_data:
	    return json.load(json_data)


def fetchModelsOP():
	with open(B_opfile_location) as json_data:
	    B_op = json.load(json_data)
	with open(A_opfile_location) as json_data:
	    A_op = json.load(json_data)
	with open(R_opfile_location) as json_data:
	    R_op = json.load(json_data)
	return B_op, A_op, R_op


def writeJSON(data, fileName):
	with open(fileName+'.json', 'w') as outfile:
		json.dump(data, outfile)


def main():
	data = fetchDevData()
	B_op, A_op, R_op = fetchModelsOP()
	print("Starting Ensemble Classifier: ")
	ec = EnsembleClassifier("MAX_VOTING")
	final_op1, final_voter1 = ec.predict1(B_op, A_op, R_op)
	writeJSON(final_op1, "word2vec-aug_final_predictions1")
	writeJSON(final_voter1, "word2vec-aug_final_predictionsVoter1")
	print("Predictions written to file final_predictions1.json")
	final_op2, final_voter2 = ec.predict2(B_op, A_op, R_op)
	writeJSON(final_op2, "word2vec-aug_final_predictions2")
	writeJSON(final_voter2, "word2vec-aug_final_predictionsVoter2")
	print("Predictions written to file final_predictions2.json")


main()

Overwriting Word2vec-aug.py


In [ ]:
!python Word2vec-aug.py

Starting Ensemble Classifier: 
Predictions written to file final_predictions1.json
Predictions written to file final_predictions2.json


In [ ]:
%cp /content/word2vec-aug_final_predictions1.json /content/drive/MyDrive/QA/model_results
%cp /content/word2vec-aug_final_predictions2.json /content/drive/MyDrive/QA/model_results
%cp /content/word2vec-aug_final_predictionsVoter1.json /content/drive/MyDrive/QA/model_results
%cp /content/word2vec-aug_final_predictionsVoter2.json /content/drive/MyDrive/QA/model_results

In [ ]:
!python evaluate-v2.0.py dev-v2.0.json word2vec-aug_final_predictions1.json

{
  "exact": 68.96319380106124,
  "f1": 72.04228727624667,
  "total": 11873,
  "HasAns_exact": 77.76653171390014,
  "HasAns_f1": 83.93354872315727,
  "HasAns_total": 5928,
  "NoAns_exact": 60.18502943650126,
  "NoAns_f1": 60.18502943650126,
  "NoAns_total": 5945
}


In [ ]:
!python evaluate-v2.0.py dev-v2.0.json word2vec-aug_final_predictions2.json

{
  "exact": 67.0765602627811,
  "f1": 70.53678584248823,
  "total": 11873,
  "HasAns_exact": 71.87921727395411,
  "HasAns_f1": 78.80959148243281,
  "HasAns_total": 5928,
  "NoAns_exact": 62.287636669470146,
  "NoAns_f1": 62.287636669470146,
  "NoAns_total": 5945
}


In [7]:
%%writefile roberta-base.py

import pandas as pd
import numpy as np
import json
from EnsembleClassifier import EnsembleClassifier


devfile_location = '/content/drive/MyDrive/QA/dev-v2.0.json'
B_opfile_location = '/content/drive/MyDrive/QA/model_results/roberta-base/squad_v2/eval_predictions.json'
A_opfile_location = '/content/drive/MyDrive/QA/model_results/roberta-base/back-trans-aug/eval_predictions.json'
R_opfile_location = '/content/drive/MyDrive/QA/model_results/roberta-base/back-trans-synonym-aug/eval_predictions.json'


def fetchDevData():
	with open(B_opfile_location) as json_data:
	    return json.load(json_data)


def fetchModelsOP():
	with open(B_opfile_location) as json_data:
	    B_op = json.load(json_data)
	with open(A_opfile_location) as json_data:
	    A_op = json.load(json_data)
	with open(R_opfile_location) as json_data:
	    R_op = json.load(json_data)
	return B_op, A_op, R_op


def writeJSON(data, fileName):
	with open(fileName+'.json', 'w') as outfile:
		json.dump(data, outfile)


def main():
	data = fetchDevData()
	B_op, A_op, R_op = fetchModelsOP()
	print("Starting Ensemble Classifier: ")
	ec = EnsembleClassifier("MAX_VOTING")
	final_op1, final_voter1 = ec.predict1(B_op, A_op, R_op)
	writeJSON(final_op1, "roberta_final_predictions1")
	writeJSON(final_voter1, "roberta-aug_final_predictionsVoter1")
	print("Predictions written to file final_predictions1.json")
	final_op2, final_voter2 = ec.predict2(B_op, A_op, R_op)
	writeJSON(final_op2, "roberta_final_predictions2")
	writeJSON(final_voter2, "roberta_final_predictionsVoter2")
	print("Predictions written to file final_predictions2.json")


main()

Writing roberta-base.py


In [8]:
!python roberta-base.py

Starting Ensemble Classifier: 
Predictions written to file final_predictions1.json
Predictions written to file final_predictions2.json


In [9]:
!python evaluate-v2.0.py dev-v2.0.json roberta_final_predictions1.json

{
  "exact": 80.08085572306915,
  "f1": 83.0778486198728,
  "total": 11873,
  "HasAns_exact": 79.84143049932524,
  "HasAns_f1": 85.84401090819014,
  "HasAns_total": 5928,
  "NoAns_exact": 80.31959629941127,
  "NoAns_f1": 80.31959629941127,
  "NoAns_total": 5945
}


In [10]:
!python evaluate-v2.0.py dev-v2.0.json roberta_final_predictions2.json

{
  "exact": 80.40933209803757,
  "f1": 83.68907709533843,
  "total": 11873,
  "HasAns_exact": 77.27732793522267,
  "HasAns_f1": 83.84622340636886,
  "HasAns_total": 5928,
  "NoAns_exact": 83.53238015138773,
  "NoAns_f1": 83.53238015138773,
  "NoAns_total": 5945
}


In [20]:
%%writefile roberta-base-BT-E.py

import pandas as pd
import numpy as np
import json
from EnsembleClassifier import EnsembleClassifier


devfile_location = '/content/drive/MyDrive/QA/dev-v2.0.json'
B_opfile_location = '/content/drive/MyDrive/QA/model_results/roberta-base/squad_v2/eval_predictions.json'
A_opfile_location = '/content/drive/MyDrive/QA/model_results/roberta-base/back-trans-aug/eval_predictions.json'
R_opfile_location = '/content/drive/MyDrive/QA/model_results/roberta-base/word2vec-aug/eval_predictions.json'


def fetchDevData():
	with open(B_opfile_location) as json_data:
	    return json.load(json_data)


def fetchModelsOP():
	with open(B_opfile_location) as json_data:
	    B_op = json.load(json_data)
	with open(A_opfile_location) as json_data:
	    A_op = json.load(json_data)
	with open(R_opfile_location) as json_data:
	    R_op = json.load(json_data)
	return B_op, A_op, R_op


def writeJSON(data, fileName):
	with open(fileName+'.json', 'w') as outfile:
		json.dump(data, outfile)


def main():
	data = fetchDevData()
	B_op, A_op, R_op = fetchModelsOP()
	print("Starting Ensemble Classifier: ")
	ec = EnsembleClassifier("MAX_VOTING")
	final_op1, final_voter1 = ec.predict1(B_op, A_op, R_op)
	writeJSON(final_op1, "roberta-BT-E_final_predictions1")
	writeJSON(final_voter1, "roberta-BT-E_final_predictionsVoter1")
	print("Predictions written to file final_predictions1.json")
	final_op2, final_voter2 = ec.predict2(B_op, A_op, R_op)
	writeJSON(final_op2, "roberta-BT-E_final_predictions2")
	writeJSON(final_voter2, "roberta-BT-E_final_predictionsVoter2")
	print("Predictions written to file final_predictions2.json")


main()

Overwriting roberta-base-BT-E.py


In [21]:
!python roberta-base-BT-E.py

Starting Ensemble Classifier: 
Predictions written to file final_predictions1.json
Predictions written to file final_predictions2.json


In [22]:
!python evaluate-v2.0.py dev-v2.0.json roberta-BT-E_final_predictions1.json

{
  "exact": 79.71868946348859,
  "f1": 82.76883503820255,
  "total": 11873,
  "HasAns_exact": 79.77395411605939,
  "HasAns_f1": 85.88299230913957,
  "HasAns_total": 5928,
  "NoAns_exact": 79.66358284272498,
  "NoAns_f1": 79.66358284272498,
  "NoAns_total": 5945
}


In [23]:
!python evaluate-v2.0.py dev-v2.0.json roberta-BT-E_final_predictions2.json

{
  "exact": 80.0218984249979,
  "f1": 83.2871485623291,
  "total": 11873,
  "HasAns_exact": 77.19298245614036,
  "HasAns_f1": 83.73284663976628,
  "HasAns_total": 5928,
  "NoAns_exact": 82.84272497897393,
  "NoAns_f1": 82.84272497897393,
  "NoAns_total": 5945
}


In [3]:
%%writefile roberta-base-BT-S.py

import pandas as pd
import numpy as np
import json
from EnsembleClassifier import EnsembleClassifier


devfile_location = '/content/drive/MyDrive/QA/dev-v2.0.json'
B_opfile_location = '/content/drive/MyDrive/QA/model_results/roberta-base/squad_v2/eval_predictions.json'
A_opfile_location = '/content/drive/MyDrive/QA/model_results/roberta-base/back-trans-aug/eval_predictions.json'
R_opfile_location = '/content/drive/MyDrive/QA/model_results/roberta-base/synonym-aug/eval_predictions.json'


def fetchDevData():
	with open(B_opfile_location) as json_data:
	    return json.load(json_data)


def fetchModelsOP():
	with open(B_opfile_location) as json_data:
	    B_op = json.load(json_data)
	with open(A_opfile_location) as json_data:
	    A_op = json.load(json_data)
	with open(R_opfile_location) as json_data:
	    R_op = json.load(json_data)
	return B_op, A_op, R_op


def writeJSON(data, fileName):
	with open(fileName+'.json', 'w') as outfile:
		json.dump(data, outfile)


def main():
	data = fetchDevData()
	B_op, A_op, R_op = fetchModelsOP()
	print("Starting Ensemble Classifier: ")
	ec = EnsembleClassifier("MAX_VOTING")
	final_op1, final_voter1 = ec.predict1(B_op, A_op, R_op)
	writeJSON(final_op1, "roberta-BT-S_final_predictions1")
	writeJSON(final_voter1, "roberta-BT-S_final_predictionsVoter1")
	print("Predictions written to file final_predictions1.json")
	final_op2, final_voter2 = ec.predict2(B_op, A_op, R_op)
	writeJSON(final_op2, "roberta-BT-S_final_predictions2")
	writeJSON(final_voter2, "roberta-BT-S_final_predictionsVoter2")
	print("Predictions written to file final_predictions2.json")


main()

Writing roberta-base-BT-S.py


In [4]:
!python roberta-base-BT-S.py

Starting Ensemble Classifier: 
Predictions written to file final_predictions1.json
Predictions written to file final_predictions2.json


In [5]:
!python evaluate-v2.0.py dev-v2.0.json roberta-BT-S_final_predictions1.json

{
  "exact": 80.24088267497685,
  "f1": 83.30864847696463,
  "total": 11873,
  "HasAns_exact": 79.25101214574899,
  "HasAns_f1": 85.39534132371823,
  "HasAns_total": 5928,
  "NoAns_exact": 81.22792262405383,
  "NoAns_f1": 81.22792262405383,
  "NoAns_total": 5945
}


In [6]:
!python evaluate-v2.0.py dev-v2.0.json roberta-BT-S_final_predictions2.json

{
  "exact": 80.510401751874,
  "f1": 83.778605784183,
  "total": 11873,
  "HasAns_exact": 77.05802968960863,
  "HasAns_f1": 83.60381013421153,
  "HasAns_total": 5928,
  "NoAns_exact": 83.9529015979815,
  "NoAns_f1": 83.9529015979815,
  "NoAns_total": 5945
}


In [11]:
%%writefile roberta-base-BT-S-E.py

import pandas as pd
import numpy as np
import json
from EnsembleClassifier import EnsembleClassifier


devfile_location = '/content/drive/MyDrive/QA/dev-v2.0.json'
B_opfile_location = '/content/drive/MyDrive/QA/model_results/roberta-base/synonym-aug/eval_predictions.json'
A_opfile_location = '/content/drive/MyDrive/QA/model_results/roberta-base/back-trans-aug/eval_predictions.json'
R_opfile_location = '/content/drive/MyDrive/QA/model_results/roberta-base/word2vec-aug/eval_predictions.json'


def fetchDevData():
	with open(B_opfile_location) as json_data:
	    return json.load(json_data)


def fetchModelsOP():
	with open(B_opfile_location) as json_data:
	    B_op = json.load(json_data)
	with open(A_opfile_location) as json_data:
	    A_op = json.load(json_data)
	with open(R_opfile_location) as json_data:
	    R_op = json.load(json_data)
	return B_op, A_op, R_op


def writeJSON(data, fileName):
	with open(fileName+'.json', 'w') as outfile:
		json.dump(data, outfile)


def main():
	data = fetchDevData()
	B_op, A_op, R_op = fetchModelsOP()
	print("Starting Ensemble Classifier: ")
	ec = EnsembleClassifier("MAX_VOTING")
	final_op1, final_voter1 = ec.predict1(B_op, A_op, R_op)
	writeJSON(final_op1, "roberta-BT-S-E_final_predictions1")
	writeJSON(final_voter1, "roberta-BT-S-E_final_predictionsVoter1")
	print("Predictions written to file final_predictions1.json")
	final_op2, final_voter2 = ec.predict2(B_op, A_op, R_op)
	writeJSON(final_op2, "roberta-BT-S-E_final_predictions2")
	writeJSON(final_voter2, "roberta-BT-S-E_final_predictionsVoter2")
	print("Predictions written to file final_predictions2.json")


main()

Writing roberta-base-BT-S-E.py


In [12]:
!python roberta-base-BT-S-E.py

Starting Ensemble Classifier: 
Predictions written to file final_predictions1.json
Predictions written to file final_predictions2.json


In [13]:
!python evaluate-v2.0.py dev-v2.0.json roberta-BT-S-E_final_predictions1.json

{
  "exact": 78.59007832898172,
  "f1": 81.56828738530444,
  "total": 11873,
  "HasAns_exact": 79.99325236167341,
  "HasAns_f1": 85.95821122228752,
  "HasAns_total": 5928,
  "NoAns_exact": 77.19091673675358,
  "NoAns_f1": 77.19091673675358,
  "NoAns_total": 5945
}


In [14]:
!python evaluate-v2.0.py dev-v2.0.json roberta-BT-S-E_final_predictions2.json

{
  "exact": 78.2616019540133,
  "f1": 81.35734046311282,
  "total": 11873,
  "HasAns_exact": 77.68218623481782,
  "HasAns_f1": 83.88254104563731,
  "HasAns_total": 5928,
  "NoAns_exact": 78.83936080740118,
  "NoAns_f1": 78.83936080740118,
  "NoAns_total": 5945
}


In [16]:
%%writefile En2+3+4.py

import pandas as pd
import numpy as np
import json
from EnsembleClassifier import EnsembleClassifier


devfile_location = '/content/drive/MyDrive/QA/dev-v2.0.json'
B_opfile_location = '/content/roberta-BT-S_final_predictions2.json'
A_opfile_location = '/content/roberta_final_predictions2.json'
R_opfile_location = '/content/roberta-BT-S-E_final_predictions1.json'


def fetchDevData():
	with open(B_opfile_location) as json_data:
	    return json.load(json_data)


def fetchModelsOP():
	with open(B_opfile_location) as json_data:
	    B_op = json.load(json_data)
	with open(A_opfile_location) as json_data:
	    A_op = json.load(json_data)
	with open(R_opfile_location) as json_data:
	    R_op = json.load(json_data)
	return B_op, A_op, R_op


def writeJSON(data, fileName):
	with open(fileName+'.json', 'w') as outfile:
		json.dump(data, outfile)


def main():
	data = fetchDevData()
	B_op, A_op, R_op = fetchModelsOP()
	print("Starting Ensemble Classifier: ")
	ec = EnsembleClassifier("MAX_VOTING")
	final_op1, final_voter1 = ec.predict1(B_op, A_op, R_op)
	writeJSON(final_op1, "En5_final_predictions1")
	writeJSON(final_voter1, "En5_final_predictionsVoter1")
	print("Predictions written to file final_predictions1.json")
	final_op2, final_voter2 = ec.predict2(B_op, A_op, R_op)
	writeJSON(final_op2, "En5_final_predictions2")
	writeJSON(final_voter2, "En5_final_predictionsVoter2")
	print("Predictions written to file final_predictions2.json")


main()

Overwriting En2+3+4.py


In [17]:
!python En2+3+4.py

Starting Ensemble Classifier: 
Predictions written to file final_predictions1.json
Predictions written to file final_predictions2.json


In [18]:
!python evaluate-v2.0.py dev-v2.0.json En5_final_predictions1.json

{
  "exact": 80.56093657879222,
  "f1": 83.81911872118376,
  "total": 11873,
  "HasAns_exact": 76.95681511470985,
  "HasAns_f1": 83.48252303924022,
  "HasAns_total": 5928,
  "NoAns_exact": 84.15475189234651,
  "NoAns_f1": 84.15475189234651,
  "NoAns_total": 5945
}


In [19]:
!python evaluate-v2.0.py dev-v2.0.json En5_final_predictions2.json

{
  "exact": 80.56093657879222,
  "f1": 83.81911872118376,
  "total": 11873,
  "HasAns_exact": 76.95681511470985,
  "HasAns_f1": 83.48252303924022,
  "HasAns_total": 5928,
  "NoAns_exact": 84.15475189234651,
  "NoAns_f1": 84.15475189234651,
  "NoAns_total": 5945
}
